In [5]:
import sim
import numpy as np
import time as t

#### Instrucciones

        simRemoteApi.start(20000)

In [6]:
def connect(puerto):
    sim.simxFinish(-1)
    clienteID = sim.simxStart('127.0.0.1', puerto, True, True, 2000, 5)
    if(clienteID == 0): print('conectado a ', puerto)
    else: print('no se pudo conectar')
    return clienteID


In [25]:
clientID = connect(20000)

conectado a  20000


#### Obtener los manejadores (handlers)


In [8]:
#manejador dummy
returnCode, handle = sim.simxGetObjectHandle(clientID, 'Dummy',sim.simx_opmode_blocking)
dummy= handle
print(dummy)

92


In [9]:
#a partir de su menejador podemos accionar sobre el objeto
#por ejemplo obtener su posicion
returnCode, Dummy = sim.simxGetObjectPosition(clientID, dummy,-1,  sim.simx_opmode_blocking)
print(Dummy)

[6.685073458356783e-06, 2.55399318120908e-05, 1.4559825658798218]


In [10]:
#obtenemos los manejadores para cada una de las articulaciones del robot
ret, joint0 = sim.simxGetObjectHandle(clientID, 'joint_0',sim.simx_opmode_blocking)
ret, joint1 = sim.simxGetObjectHandle(clientID, 'joint_1',sim.simx_opmode_blocking)
ret, joint2 = sim.simxGetObjectHandle(clientID, 'joint_2',sim.simx_opmode_blocking)
ret, joint3 = sim.simxGetObjectHandle(clientID, 'joint_3',sim.simx_opmode_blocking)

ret, rotacion_garra = sim.simxGetObjectHandle(clientID, 'rotacion_garra',sim.simx_opmode_blocking)
ret, garra_1 = sim.simxGetObjectHandle(clientID, 'garra_1',sim.simx_opmode_blocking)
ret, garra_2 = sim.simxGetObjectHandle(clientID, 'garra_2',sim.simx_opmode_blocking)


#### Cual es la posición de las articulaciones?
Utilizando los manejadores, podemos obtener información de los elementos.

In [11]:
returnCode, pos0 = sim.simxGetJointPosition(clientID, joint0, sim.simx_opmode_blocking)
returnCode, pos1 = sim.simxGetJointPosition(clientID, joint1, sim.simx_opmode_blocking)
print(pos0, pos1)

-7.820215728315816e-07 1.5707957744598389


#### .... y movemos el robot
Utilizando los manejadores, podemos enviar parámetros a los elementos


In [26]:
def set_ang(join_n, ang):
    sim.simxSetJointTargetPosition(clientID, join_n, np.deg2rad(ang), sim.simx_opmode_oneshot)

set_ang(joint3, ang = 45)   
set_ang(joint2, ang = 10)   
set_ang(joint1, ang = 90)

for ang in np.linspace(0, 360, 180):
    t.sleep(.001)
    set_ang(joint0, ang/2)
    set_ang(joint1, -ang*.3 + 90)
    #set_ang(joint2, ang/4 )  
print('terminado')

terminado
